In [1]:
import pandas as pd
import cleaningtools as ct


In [2]:
assays=pd.read_csv('../../_Master Databases/Master_Spectral_Clean_2022_06_29.csv')
spectral=pd.read_csv('../../_Master Databases/Master_Assay_Clean_2022_06_29.csv',low_memory=False)

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_3415/2526389175.py:1: DtypeWarning: Columns (110) have mixed types. Specify dtype option on import or set low_memory=False.
  assays=pd.read_csv('../../_Master Databases/Master_Spectral_Clean_2022_06_29.csv')


In [4]:
assays.head()

,sample_id,Spectrum,Company,Country,Project,DispatchID,LabJobNumber,PONumber,HoleID,SpectromNo,...,Int_FeOxide,Wav_FeOxide,D_FeOxide,FIntFeOxide,Recvd Wt.,SpectrumId,start_depth,end_depth,file,folder
0,B300001,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9302,NaN,NaN,NaN,2.14,Available,nan0.0,1.0nan,nanTU20155409.csv,TU20155409nan
1,B300002,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9270,NaN,NaN,NaN,6.22,Available,nan1.0,2.0nan,nanTU20155409.csv,TU20155409nan
2,B300003,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9314,NaN,NaN,NaN,5.05,Available,nan2.0,3.0nan,nanTU20155409.csv,TU20155409nan
3,B300004,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9136,NaN,NaN,NaN,4.22,Available,nan3.0,4.0nan,nanTU20155409.csv,TU20155409nan
4,B300005,Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9106,NaN,NaN,NaN,4.82,Available,nan4.0,5.0nan,nanTU20155409.csv,TU20155409nan
